In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
tf.get_logger().setLevel(tf.compat.v1.logging.ERROR)

import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score


In [ ]:
df=pd.read_csv("/home/vinod/asetiya/train_test_data_10449_bonferroni.csv")

In [ ]:
df.reset_index(drop=True, inplace=True)
df.fillna(0,inplace=True)
df.replace([np.inf, -np.inf], 0, inplace=True)


In [ ]:
y=df['Toxicity']
X=df.drop(['Toxicity'],axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Get the indices of the train and test sets
train_indices = X_train.index
test_indices = X_test.index

In [ ]:
from sklearn.preprocessing import StandardScaler

# Select only the numerical columns
numerical_columns = X.columns[(X.dtypes == 'float64') & (X.columns != 'IPC')]


# Create a DataFrame with only the numerical columns
numerical_X = X[numerical_columns]

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the numerical data and transform it for the train set
scaled_numerical_X_train = scaler.fit_transform(numerical_X.loc[X_train.index])

# Replace the scaled numerical data back into the original DataFrame for the train set
X.loc[X_train.index, numerical_columns] = scaled_numerical_X_train

# Transform the numerical data for the test set
scaled_numerical_X_test = scaler.transform(numerical_X.loc[X_test.index])

# Replace the scaled numerical data back into the original DataFrame for the test set
X.loc[X_test.index, numerical_columns] = scaled_numerical_X_test


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Convert the data to the appropriate data types
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)
from sklearn.model_selection import StratifiedKFold
import numpy as np
from bayes_opt import BayesianOptimization
import tensorflow as tf

from sklearn.utils.class_weight import compute_class_weight


# Assuming y_train is your target variable
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Convert class weights to a dictionary
class_weight_dict = dict(enumerate(class_weights))


# Assuming X_train, y_train are already defined
# You may need to load your dataset before running this code

# Define the MLP architecture using TensorFlow
def create_model(learning_rate, dropout_rate,regularization_strength):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(1024, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=tf.keras.regularizers.l2(regularization_strength)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(regularization_strength)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(regularization_strength)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])


    # Compile the model with the given learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define the parameter search space
# Define the parameter search space
pbounds = {
    'learning_rate': (0.001, 0.1),
    'dropout_rate': (0.2, 0.5),

    'regularization_strength': (1e-5, 0.01),  # Range for L2 regularization strength
}


# Define the function to optimize with cross-validation
def optimize_model_cv(learning_rate, dropout_rate,  regularization_strength):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    val_accuracies = []

    for train_index, val_index in skf.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        model = create_model(learning_rate, dropout_rate,  regularization_strength)
        history = model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, verbose=0,
                            class_weight=class_weight_dict, validation_data=(X_val_fold, y_val_fold))
        val_accuracies.append(history.history['val_accuracy'][-1])

    average_val_accuracy = np.mean(val_accuracies)
    return average_val_accuracy


# Perform Bayesian optimization
optimizer = BayesianOptimization(f=optimize_model_cv, pbounds=pbounds)
optimizer.maximize(init_points=5, n_iter=5)

# Print the best hyperparameters and corresponding accuracy
print("Best Hyperparameters: ", optimizer.max['params'])
print("Best Average Validation Accuracy: ", optimizer.max['target'])


In [ ]:
# Print the best hyperparameters and corresponding accuracy
print("Best Hyperparameters: ", optimizer.max['params'])
print("Best Average Validation Accuracy: ", optimizer.max['target'])